# Windows in Point Patterns

**Author: Serge Rey <sjsrey@gmail.com>**

## Introduction
Windows play several important roles in the analysis of planar point patterns. As we saw briefly in the [introductory notebook](pointpattern.ipynb), the area of the window can be used to develop estimates of the intensity of the point pattern. A window also defines the domain for the point pattern and can support corrections for so-called edge effects in the statistical analysis of point patterns. However, their are different ways to define a window for a point pattern.

This notebook provides an overview of how to work with windows and covers the following:

* [Creating a window](#Creating-a-Window)
* [Window attributes](#Window-Attributes)
* [Window methods](#Window-Methods)
* [Multi-part windows](#Multi-part-Windows)


## Creating a Window

We will first continue on with an example from the [introductory notebook](pointpattern.ipynb). Recall this uses  200 randomly distributed points within the counties of Virginia. Coordinates are for UTM zone 17 N.

In [25]:
import pysal as ps
import numpy as np
from pysal.contrib.points.pointpattern import PointPattern

In [26]:
f = ps.examples.get_path('vautm17n_points.shp')
fo = ps.open(f)
pp_va = PointPattern(np.asarray([pnt for pnt in fo]))
fo.close()
pp_va.summary()

Point Pattern
200 points
Bounding rectangle [(273959.664381,4049220.90341), (972595.989578,4359604.85978)]
Area of window: 2.16845506675e+11
Intensity estimate for window: 9.22315629531e-10
               x               y
0  865322.486181  4150317.315954
1  774479.213103  4258993.023041
2  308048.692232  4054699.778123
3  670711.529980  4258863.858165
4  666254.475614  4256514.364659


From the summary method we see that the Bounding Rectangle is reported along with the Area of the window for the point pattern. Two things to note here. First, the only argument we passed in to the `PointPattern`s constructor was the array of coordinates for the 200 points. In this case PySAL finds the [minimum bounding box](https://en.wikipedia.org/wiki/Minimum_bounding_rectangle) for the point pattern and uses this as the window.

The second thing to note is that the area of the window in this case is simply the area of the bounding rectangle. Because we are using projected coordinates (UTM) the unit of measure for the area is in square meters.

## Window Attributes

The window is an attribute of the `PointPattern`. It is also an object with its own attributes:

In [9]:
pp_va.window.area

216845506675.05569

In [10]:
pp_va.window.bbox

[273959.664381352, 4049220.903414295, 972595.9895779632, 4359604.85977962]

The bounding box is given in left, bottom, right, top ordering.

In [12]:
pp_va.window.centroid

(623277.82697965798, 4204412.8815969583)

In [21]:
pp_va.window.parts

[[(273959.66438135202, 4049220.9034142951),
  (273959.66438135202, 4359604.8597796196),
  (972595.98957796325, 4359604.8597796196),
  (972595.98957796325, 4049220.9034142951),
  (273959.66438135202, 4049220.9034142951)]]

The `parts` attribute for the `window` is a list of polygons. In this case the window has only a single part and it is a rectangular polygon with vertices listed clockwise in closed cartographic form.

## Window Methods

A window has several basic geometric operations that are heavily used in some of the other modules in the the `Point` package, but that can also be used directly.

In [22]:
pp_va.window.contains_point((623277.82697965798, 4204412.8815969583))

True

In [23]:
pnts = ((-623277.82697965798, 4204412.8815969583),
        (623277.82697965798, 4204412.8815969583),
        (1000.01, 200.9))

In [24]:
pnts_in = pp_va.window.filter_contained(pnts)
pnts_in

[array([  623277.82697966,  4204412.88159696])]

In [11]:
#from shapely.geometry import LinearRing, Point, Polygon, MultiPolygon
from pysal.contrib import shapely_ext
import numpy as np
from window import poly_from_bbox, as_window, Window
import pysal as ps
%matplotlib inline
import matplotlib.pyplot as plt

In [12]:
parts = [[(0.0, 0.0), (0.0, 10.0), (10.0, 10.0), (10.0, 0.0)],
         [(11.,11.), (11.,20.), (20.,20.), (20.,11.)]]
holes = [[(3.0,3.0), (6.0, 3.0), (6.0, 6.0), (3.0, 6.0)]]


In [13]:
window = Window(parts, holes)
window.bbox

[0.0, 0.0, 20.0, 20.0]

In [14]:
window.area

172.0

In [15]:
window.contains_point((12,12))

True

In [16]:
window.contains_point((4,4))

False

In [17]:
window.contains_point((2,2))

True

In [18]:
pin = window.filter_contained([(12,12), (4,4), (2,2)])

In [19]:
pin

[array([12, 12]), array([2, 2])]

In [20]:
va = ps.open(ps.examples.get_path("virginia.shp"))
polys = [shp for shp in va]
vapnts = ps.open(ps.examples.get_path("vapoints.shp"))
points = [shp for shp in vapnts]

In [21]:
from pysal.contrib import shapely_ext

In [22]:
cu = shapely_ext.cascaded_union(polys)

In [23]:
cu

In [24]:
w = as_window(cu)

In [25]:
len(w.parts)

3

In [26]:
w.bbox

[-83.67526245117188, 36.541481018066406, -75.24258422851562, 39.45690155029297]

In [28]:
w.centroid

(-78.85183583334933, 37.51851209850039)

In [29]:
w.contains_point(w.centroid)

True

In [30]:
# find which county the state centroid is in using county windows
windows = [as_window(county) for county in polys]

In [32]:
cent_poly = [ (i, county) for i,county in enumerate(windows) if county.contains_point(w.centroid)]

In [33]:
cent_poly

[(67, <window.Window at 0x10b326690>)]

In [35]:
i, cent_poly = cent_poly[0]

In [36]:
cent_poly.bbox

[-79.02201843261719, 37.19912338256836, -78.59026336669922, 37.55881118774414]

In [37]:
i

67

##Multi-part Windows